# 数据处理
先转换为单题的单个字符一列的excel；只提取出有效值；例如frame\":{\"successRate\":,\"minJumps\":,\"jumps\，这些说明字段删去；
# 数据分析
1、每个题的平均作答时长；——结果数据</br>

2、每个题的编码种类（有多少种，分别是什么，每种多少学生）；——结果数据</br>

3、每个题的每个操作步骤的平均作答时长；——过程数据</br>

4、正确率（暂未提供标准编码，可以探索一下题目本身，协助形成标准答案编码）；——结果数据</br>

5、关键节点（通过数据，探索学生在从初始状态向终止状态进行的过程中，有几个关键步骤，每个关键步骤有几种类型的关键节点编码），体现“用数据说话”去探索关键节点。——过程数据</br>
【以上仅供参考，希望包含，但不限于此】

答案解析：一个frame字段是一个题。
每个题目的记录方式不同，请用demo账号登录题目，页面最上方点击打开答案，可以看到该题目的每个步骤点击后出现的答案数据，也可以直接与命题人沟通询问具体每个答案的内涵。


["{\"frame\":{\"successRate\":1,\"minJumps\":2,\"jumps\":2,\"path\":[1,3,2]}}","{\"frame\":{\"successRate\":1,\"minJumps\":2,\"jumps\":2,\"path\":[3,1,0]}}","{\"frame\":[\"00\",\"01\",\"02\",\"06\"]}","{\"frame\":[\"00\",\"01\",\"02\",\"03\"]}","{\"frame\":[[0,0,0,0,0,0,0],[1,0,1,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,1,0,0,0,1]]}","{\"frame\":[\"C_B\"]}","{\"frame\":[\"C_G\",\"F_G\"]}","{\"frame\":[\"09_02\"]}","{\"frame\":[\"v_10_v_2\"]}","{\"frame\":[]}","{\"frame\":[]}","{\"frame\":[[[2,1],[3,1],[4,1],[4,2]]]}","{\"frame\":[[{\"row\":0,\"col\":0}]]}","{\"frame\":[[{\"row\":1,\"col\":0}]]}","{\"frame\":{\"stamps\":[2,1,0],\"start\":1,\"end\":16,\"selected\":true}}","{\"frame\":{\"stamps\":[2,1,0],\"start\":1,\"end\":5,\"selected\":true}}","{\"frame\":{\"stamps\":[2,1,0,1],\"start\":5,\"end\":5,\"selected\":true}}","{\"frame\":{\"rotation\":[[0,1,0]],\"lowered\":null}}","{\"frame\":{\"rotation\":[[0,1,0,1,3,2]],\"lowered\":null}}","{\"frame\":{\"rotation\":[[null,null,2,0,2,3],[0,2,2,null,3,null],[null,0,null,null,1,null],[0,2,0,1,null,null]],\"lowered\":null}}","{\"frame\":{\"sequence\":[0,0,0,0,0,1,1],\"transformation\":[[0],[0,1]]}}","{\"frame\":{\"sequence\":[0,0,0],\"transformation\":[[0,1],[1,0]]}}","{\"frame\":[4,5]}","{\"frame\":[0,1,2,2,0,0]}"]


In [4]:
import pandas as pd
import json 
import numpy as np
import ast
from datetime import datetime
import plotly as py
import plotly.graph_objs as go
from plotly.offline import plot
from IPython.core.display import HTML
import plotly.offline as offline
offline.init_notebook_mode(connected=True)


In [89]:
class data_analysis:
    def __init__(self, df) -> None:
        self.problem_num = 23
        self.df = df
        self.row_num = len(df)
        self.df.insert(len(self.df.columns), 'ans', self.remove_str())
        self.df.insert(len(self.df.columns), 'interval', self.get_interval())
        self.ndf = pd.DataFrame(self.create_new_df())
        self.ndf_list = self.divide_ndf()
        self.group_list = self.group_by()
        print('init complete')

    def remove_str_per_row(self, data_per_row):
        frame_list = ast.literal_eval(data_per_row)
        frame_dic_list = []
        for index in range(len(frame_list)):
            frame_dic_list.append(json.loads(frame_list[index])) 
        return frame_dic_list

    def remove_str(self):
        ndf_ans_8_list = []
        ndf_rm_frame = []
        for i in range(self.row_num):
            dic_temp = self.remove_str_per_row(self.df.loc[i,'task_answers'])
            ndf_ans_8_list.append(dic_temp)
            new_dic_list = []
            for dic in dic_temp:
                dic = dic['frame']
                new_dic = dic
                new_dic_list.append(new_dic)
            ndf_rm_frame.append(new_dic_list)

        return ndf_rm_frame
    
    def get_interval(self):
        interval_list = []
        for i in range(len(self.df)):
            interval_list.append(self.get_interval_per_row(i))
        return interval_list

    def get_interval_per_row(self, index):
        row_data = self.df.loc[index,:]
        start_time = row_data['start_time']
        start_time = datetime.strptime(start_time,"%Y-%m-%dT%H:%M:%S+08:00")

        expire_time = row_data['expire_time']
        expire_time = datetime.strptime(expire_time,"%Y-%m-%dT%H:%M:%S+08:00")

        stop_time = row_data['stop_time']
        if stop_time != stop_time:
            return -1
        stop_time = datetime.strptime(stop_time,"%Y-%m-%dT%H:%M:%S+08:00")

        total_sec = (stop_time - start_time).seconds
        return total_sec
    
    def create_new_df(self):
        twoD_list = []
        for row in range(self.row_num):
            ans_dic_list = self.df.loc[row, 'ans']
            twoD_list.append(ans_dic_list)
        return twoD_list
    
    def divide_ndf(self):
        ndf_list = []
        for i in range(len(self.ndf.columns)):
            ndf_list.append(pd.DataFrame(self.ndf.loc[:,i]))
        return ndf_list
    
    def group_by_per_problem(self, index):
        df_temp = self.ndf_list[index]
        df_str_list = []
        for j in range(len(df_temp)):
            ndf_index_j = df_temp.iloc[j, 0]
            if ndf_index_j == None:
                df_str_list.append(str(None))
            else:
                df_str_list.append(self.content_to_str(ndf_index_j))
        df_temp.insert(1, 'ans_str', df_str_list)
        df_per_problom = df_temp.groupby('ans_str')
        return df_per_problom

    def content_to_str(self, data):
        str_data = ''
        if data == None:
            return str(None)
        elif type(data) == type([]):
            return self.data_to_str(data)
        elif 'data' in data.keys():
            return self.data_to_str(data['data'])
        else:
            return self.data_to_str(data)

    def data_to_str(self, data):
        if type(data) == type({}):
            return str(list(data.values()))
        else:
            return str(data)

    def group_by(self):
        group_list = []
        for i in range(self.problem_num):
            df_temp = self.group_by_per_problem(i)
            group_list.append(df_temp)
        return group_list

    def plot(self):
        data = [go.Histogram(x=list(self.df.loc[:,'interval']))] 
        layout={"title": "学生用时分布", 
                                       "xaxis_title": "学生用时，单位秒",
                                       "yaxis_title": "学生个数",
                                       # x轴坐标倾斜60度
                                       "xaxis": {"tickangle": 60}
                                      }
        fig = go.Figure(data=data,layout=layout)
        plot(fig,filename="./plot/vector.html",auto_open=False,image='png',image_height=800,image_width=1500)
        offline.iplot(fig) 
        return 0

    def plot_problem(self):
        data = [go.Bar(x = list(range(self.problem_num)), y = [len(list(group)) for group in self.group_list])] 
        layout={"title": "不同题目的编码数量", 
                                       "xaxis_title": "题目编号",
                                       "yaxis_title": "编码个数",
                                       # x轴坐标倾斜60度
                                       "xaxis": {"tickangle": 60}
                                      }
        fig = go.Figure(data=data,layout=layout)
        plot(fig,filename="./plot/vector.html",auto_open=False,image='png',image_height=800,image_width=1500)
        offline.iplot(fig) 
        return 0

In [90]:
df = pd.read_excel('./data/data.xlsx')
data_entity = data_analysis(df)

init complete


In [96]:
df.sort_values('school', ascending=True, inplace= False )

,user,type,contest_id,ticket_id,start_time,expire_time,stop_time,mobile_alert,task_answers,user_agent,client_ip,school,tag,name,owner,state,is_parent,ans,interval
0,yibin/智能计算素养/310101,ticket-user,yibin/智能计算素养,310101,2021-06-24T09:44:29+08:00,2021-06-24T11:44:29+08:00,2021-06-24T10:33:05+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,125.69.167.133,区二中实验初中,"我是学生,智能计算素养,问题解决素养",杨凤,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",2916
170,yibin/智能计算素养/310707,ticket-user,yibin/智能计算素养,310707,2021-06-24T11:44:49+08:00,2021-06-24T13:44:49+08:00,2021-06-24T12:17:22+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,125.69.167.133,区二中实验初中,"我是学生,智能计算素养,问题解决素养",赵程军,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",1953
171,yibin/智能计算素养/310708,ticket-user,yibin/智能计算素养,310708,2021-06-24T11:56:51+08:00,2021-06-24T13:56:51+08:00,2021-06-24T12:33:39+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,125.69.167.133,区二中实验初中,"我是学生,智能计算素养,问题解决素养",郭超钰,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 0, ...",2208
172,yibin/智能计算素养/310709,ticket-user,yibin/智能计算素养,310709,2021-06-24T11:57:50+08:00,2021-06-24T13:57:50+08:00,2021-06-24T12:36:06+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,125.69.167.133,区二中实验初中,"我是学生,智能计算素养,问题解决素养",杨沁川,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",2296
173,yibin/智能计算素养/310710,ticket-user,yibin/智能计算素养,310710,2021-06-24T11:51:52+08:00,2021-06-24T13:51:52+08:00,2021-06-24T12:31:26+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,125.69.167.133,区二中实验初中,"我是学生,智能计算素养,问题解决素养",吕琨玮,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 0, ...",2374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,yibin/智能计算素养/411836,ticket-user,yibin/智能计算素养,411836,2021-06-24T09:39:00+08:00,2021-06-24T11:39:00+08:00,2021-06-24T11:01:48+08:00,NaN,"[""{\""frame\"":{\""successRate\"":1,\""minJumps\"":2...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",周欣语,yibin,NaN,0,"[{'successRate': 1, 'minJumps': 2, 'jumps': 2,...",4968
392,yibin/智能计算素养/411835,ticket-user,yibin/智能计算素养,411835,2021-06-24T09:21:26+08:00,2021-06-24T11:21:26+08:00,2021-06-24T10:36:22+08:00,NaN,"[""{\""frame\"":{\""successRate\"":1,\""minJumps\"":2...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",刘睿函,yibin,NaN,0,"[{'successRate': 1, 'minJumps': 2, 'jumps': 2,...",4496
391,yibin/智能计算素养/411834,ticket-user,yibin/智能计算素养,411834,2021-06-24T09:48:45+08:00,2021-06-24T11:48:45+08:00,2021-06-24T10:48:48+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",向萍,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",3603
389,yibin/智能计算素养/411832,ticket-user,yibin/智能计算素养,411832,2021-06-24T09:34:03+08:00,2021-06-24T11:34:03+08:00,2021-06-24T11:01:48+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",张伟聪,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",5265


In [105]:
grouped = df.groupby('school')
df_l = []
for value, group in grouped:
    df_l.append(group)
df_l[1]

,user,type,contest_id,ticket_id,start_time,expire_time,stop_time,mobile_alert,task_answers,user_agent,client_ip,school,tag,name,owner,state,is_parent,ans,interval
267,yibin/智能计算素养/411601,ticket-user,yibin/智能计算素养,411601,2021-06-24T09:53:01+08:00,2021-06-24T11:53:01+08:00,2021-06-24T11:04:28+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",杨玉婷,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",4287
268,yibin/智能计算素养/411602,ticket-user,yibin/智能计算素养,411602,2021-06-24T10:08:58+08:00,2021-06-24T12:08:58+08:00,2021-06-24T11:14:09+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",王怡航,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",3911
269,yibin/智能计算素养/411603,ticket-user,yibin/智能计算素养,411603,2021-06-24T10:03:21+08:00,2021-06-24T12:03:21+08:00,2021-06-24T11:12:27+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",陈杰,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",4146
270,yibin/智能计算素养/411604,ticket-user,yibin/智能计算素养,411604,2021-06-24T08:42:39+08:00,2021-06-24T10:42:39+08:00,2021-06-24T10:21:24+08:00,NaN,"[""{\""frame\"":{\""successRate\"":1,\""minJumps\"":2...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",陈园园,yibin,NaN,0,"[{'successRate': 1, 'minJumps': 2, 'jumps': 2,...",5925
271,yibin/智能计算素养/411605,ticket-user,yibin/智能计算素养,411605,2021-06-24T09:26:58+08:00,2021-06-24T11:26:58+08:00,2021-06-24T10:24:48+08:00,NaN,"[""{\""frame\"":{\""successRate\"":1,\""minJumps\"":2...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",彭俊业,yibin,NaN,0,"[{'successRate': 1, 'minJumps': 2, 'jumps': 2,...",3470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,yibin/智能计算素养/412349,ticket-user,yibin/智能计算素养,412349,2021-06-25T09:16:59+08:00,2021-06-25T11:16:59+08:00,2021-06-25T10:18:46+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",苏雨梦,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",3707
658,yibin/智能计算素养/412350,ticket-user,yibin/智能计算素养,412350,2021-06-25T09:14:35+08:00,2021-06-25T11:14:35+08:00,2021-06-25T11:02:43+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",张廷轩,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",6488
659,yibin/智能计算素养/412351,ticket-user,yibin/智能计算素养,412351,2021-06-25T09:18:02+08:00,2021-06-25T11:18:02+08:00,2021-06-25T10:05:09+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",母小玉,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",2827
660,yibin/智能计算素养/412352,ticket-user,yibin/智能计算素养,412352,2021-06-25T09:09:23+08:00,2021-06-25T11:09:23+08:00,2021-06-25T09:52:58+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",陈明英,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",2615


In [109]:
df.loc[0:266,:].to_excel('./data/junior.xlsx')

In [111]:
df.loc[267:, :].to_excel('./data/senior.xlsx')

In [113]:
df_junior = pd.read_excel('./data/junior.xlsx')
df_junior

,Unnamed: 0,user,type,contest_id,ticket_id,start_time,expire_time,stop_time,mobile_alert,task_answers,user_agent,client_ip,school,tag,name,owner,state,is_parent,ans,interval
0,0,yibin/智能计算素养/310101,ticket-user,yibin/智能计算素养,310101,2021-06-24T09:44:29+08:00,2021-06-24T11:44:29+08:00,2021-06-24T10:33:05+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,125.69.167.133,区二中实验初中,"我是学生,智能计算素养,问题解决素养",杨凤,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",2916
1,1,yibin/智能计算素养/310102,ticket-user,yibin/智能计算素养,310102,2021-06-24T09:49:17+08:00,2021-06-24T11:49:17+08:00,2021-06-24T10:10:22+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,125.69.167.133,区二中实验初中,"我是学生,智能计算素养,问题解决素养",祝韵涵,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",1265
2,2,yibin/智能计算素养/310103,ticket-user,yibin/智能计算素养,310103,2021-06-24T09:50:20+08:00,2021-06-24T11:50:20+08:00,2021-06-24T10:26:06+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,125.69.167.133,区二中实验初中,"我是学生,智能计算素养,问题解决素养",兰英,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",2146
3,3,yibin/智能计算素养/310104,ticket-user,yibin/智能计算素养,310104,2021-06-24T10:11:47+08:00,2021-06-24T12:11:47+08:00,2021-06-24T10:39:18+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,125.69.167.133,区二中实验初中,"我是学生,智能计算素养,问题解决素养",王福琳,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",1651
4,4,yibin/智能计算素养/310105,ticket-user,yibin/智能计算素养,310105,2021-06-24T09:57:54+08:00,2021-06-24T11:57:54+08:00,2021-06-24T10:34:30+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,125.69.167.133,区二中实验初中,"我是学生,智能计算素养,问题解决素养",程诗茹,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 0, ...",2196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,262,yibin/智能计算素养/310951,ticket-user,yibin/智能计算素养,310951,2021-06-25T11:19:28+08:00,2021-06-25T13:19:28+08:00,2021-06-25T12:20:53+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,125.69.167.133,区二中实验初中,"我是学生,智能计算素养,问题解决素养",孙其杰,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",3685
263,263,yibin/智能计算素养/310952,ticket-user,yibin/智能计算素养,310952,2021-06-25T11:46:57+08:00,2021-06-25T13:46:57+08:00,2021-06-25T12:32:01+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,125.69.167.133,区二中实验初中,"我是学生,智能计算素养,问题解决素养",韦天远,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",2704
264,264,yibin/智能计算素养/310953,ticket-user,yibin/智能计算素养,310953,2021-06-25T11:30:18+08:00,2021-06-25T13:30:18+08:00,2021-06-25T12:31:54+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,125.69.167.133,区二中实验初中,"我是学生,智能计算素养,问题解决素养",蒋宇杰,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",3696
265,265,yibin/智能计算素养/310954,ticket-user,yibin/智能计算素养,310954,2021-06-25T11:04:55+08:00,2021-06-25T13:04:55+08:00,2021-06-25T12:16:30+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,125.69.167.133,区二中实验初中,"我是学生,智能计算素养,问题解决素养",杨婉婷,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",4295


In [112]:
df_senior = pd.read_excel('./data/senior.xlsx')
df_senior

,Unnamed: 0,user,type,contest_id,ticket_id,start_time,expire_time,stop_time,mobile_alert,task_answers,user_agent,client_ip,school,tag,name,owner,state,is_parent,ans,interval
0,267,yibin/智能计算素养/411601,ticket-user,yibin/智能计算素养,411601,2021-06-24T09:53:01+08:00,2021-06-24T11:53:01+08:00,2021-06-24T11:04:28+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",杨玉婷,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",4287
1,268,yibin/智能计算素养/411602,ticket-user,yibin/智能计算素养,411602,2021-06-24T10:08:58+08:00,2021-06-24T12:08:58+08:00,2021-06-24T11:14:09+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",王怡航,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",3911
2,269,yibin/智能计算素养/411603,ticket-user,yibin/智能计算素养,411603,2021-06-24T10:03:21+08:00,2021-06-24T12:03:21+08:00,2021-06-24T11:12:27+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",陈杰,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",4146
3,270,yibin/智能计算素养/411604,ticket-user,yibin/智能计算素养,411604,2021-06-24T08:42:39+08:00,2021-06-24T10:42:39+08:00,2021-06-24T10:21:24+08:00,NaN,"[""{\""frame\"":{\""successRate\"":1,\""minJumps\"":2...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",陈园园,yibin,NaN,0,"[{'successRate': 1, 'minJumps': 2, 'jumps': 2,...",5925
4,271,yibin/智能计算素养/411605,ticket-user,yibin/智能计算素养,411605,2021-06-24T09:26:58+08:00,2021-06-24T11:26:58+08:00,2021-06-24T10:24:48+08:00,NaN,"[""{\""frame\"":{\""successRate\"":1,\""minJumps\"":2...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",彭俊业,yibin,NaN,0,"[{'successRate': 1, 'minJumps': 2, 'jumps': 2,...",3470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,657,yibin/智能计算素养/412349,ticket-user,yibin/智能计算素养,412349,2021-06-25T09:16:59+08:00,2021-06-25T11:16:59+08:00,2021-06-25T10:18:46+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",苏雨梦,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",3707
391,658,yibin/智能计算素养/412350,ticket-user,yibin/智能计算素养,412350,2021-06-25T09:14:35+08:00,2021-06-25T11:14:35+08:00,2021-06-25T11:02:43+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",张廷轩,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",6488
392,659,yibin/智能计算素养/412351,ticket-user,yibin/智能计算素养,412351,2021-06-25T09:18:02+08:00,2021-06-25T11:18:02+08:00,2021-06-25T10:05:09+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",母小玉,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",2827
393,660,yibin/智能计算素养/412352,ticket-user,yibin/智能计算素养,412352,2021-06-25T09:09:23+08:00,2021-06-25T11:09:23+08:00,2021-06-25T09:52:58+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",陈明英,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",2615


In [227]:
data_entity.df

,user,type,contest_id,ticket_id,start_time,expire_time,stop_time,mobile_alert,task_answers,user_agent,client_ip,school,tag,name,owner,state,is_parent,ans,interval
0,yibin/智能计算素养/310101,ticket-user,yibin/智能计算素养,310101,2021-06-24T09:44:29+08:00,2021-06-24T11:44:29+08:00,2021-06-24T10:33:05+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,125.69.167.133,区二中实验初中,"我是学生,智能计算素养,问题解决素养",杨凤,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",2916
1,yibin/智能计算素养/310102,ticket-user,yibin/智能计算素养,310102,2021-06-24T09:49:17+08:00,2021-06-24T11:49:17+08:00,2021-06-24T10:10:22+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,125.69.167.133,区二中实验初中,"我是学生,智能计算素养,问题解决素养",祝韵涵,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",1265
2,yibin/智能计算素养/310103,ticket-user,yibin/智能计算素养,310103,2021-06-24T09:50:20+08:00,2021-06-24T11:50:20+08:00,2021-06-24T10:26:06+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,125.69.167.133,区二中实验初中,"我是学生,智能计算素养,问题解决素养",兰英,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",2146
3,yibin/智能计算素养/310104,ticket-user,yibin/智能计算素养,310104,2021-06-24T10:11:47+08:00,2021-06-24T12:11:47+08:00,2021-06-24T10:39:18+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,125.69.167.133,区二中实验初中,"我是学生,智能计算素养,问题解决素养",王福琳,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",1651
4,yibin/智能计算素养/310105,ticket-user,yibin/智能计算素养,310105,2021-06-24T09:57:54+08:00,2021-06-24T11:57:54+08:00,2021-06-24T10:34:30+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,125.69.167.133,区二中实验初中,"我是学生,智能计算素养,问题解决素养",程诗茹,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 0, ...",2196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,yibin/智能计算素养/412349,ticket-user,yibin/智能计算素养,412349,2021-06-25T09:16:59+08:00,2021-06-25T11:16:59+08:00,2021-06-25T10:18:46+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",苏雨梦,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",3707
658,yibin/智能计算素养/412350,ticket-user,yibin/智能计算素养,412350,2021-06-25T09:14:35+08:00,2021-06-25T11:14:35+08:00,2021-06-25T11:02:43+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",张廷轩,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",6488
659,yibin/智能计算素养/412351,ticket-user,yibin/智能计算素养,412351,2021-06-25T09:18:02+08:00,2021-06-25T11:18:02+08:00,2021-06-25T10:05:09+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",母小玉,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",2827
660,yibin/智能计算素养/412352,ticket-user,yibin/智能计算素养,412352,2021-06-25T09:09:23+08:00,2021-06-25T11:09:23+08:00,2021-06-25T09:52:58+08:00,NaN,"[""{\""frame\"":{\""level\"":\""easy\"",\""data\"":{\""s...",Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,117.172.174.50,南溪一中（高中）,"我是学生,智能计算素养,问题解决素养,高中生问卷一,高中生问卷二",陈明英,yibin,NaN,0,"[{'level': 'easy', 'data': {'successRate': 1, ...",2615


# 数据处理
处理函数定义在类data_analysis中的子函数remove_str()中
移除每一行中第8列中的frame\":{\"successRate\":,\"minJumps\":,\"jumps\，这些说明字段删去；

In [129]:
index = 1

In [130]:
data_per_row = df.iloc[index,8]
frame_list = ast.literal_eval(data_per_row)
frame_dic_list = []
for index in range(len(frame_list)):
    frame_dic_list.append(json.loads(frame_list[index])) 
frame_dic_list

[{'frame': {'level': 'easy',
   'data': {'successRate': 1, 'minJumps': 2, 'jumps': 2, 'path': [2, 0, 1]}}},
 {'frame': {'level': 'easy',
   'data': {'successRate': 1, 'minJumps': 2, 'jumps': 2, 'path': [2, 0, 1]}}},
 {'frame': {'level': 'easy', 'data': ['00', '01', '02', '06']}},
 {'frame': {'level': 'medium', 'data': ['05', '08']}},
 {'frame': {'level': 'easy',
   'data': [[0, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0]]}},
 {'frame': {'level': 'easy', 'data': ['C_A']}},
 {'frame': {'level': 'medium', 'data': ['C_G', 'F_G']}},
 {'frame': {'level': 'easy',
   'data': [[[0, 0], [0, 1], [1, 1]],
    [[0, 2], [0, 3], [1, 3]],
    [[0, 4], [0, 5], [1, 5]],
    [[0, 6], [0, 7], [1, 7]]]}},
 {'frame': {'level': 'medium',
   'data': [[[0, 2], [1, 2], [2, 2], [2, 3]],
    None,
    [[3, 3], [4, 3], [5, 3], [5, 4]],
    [[2, 1], [3, 1], [4, 1], [4, 2]],
    [[1, 4], [2, 4], [3, 4], [3, 5]]]}},
 {'frame': {'level':

In [125]:
test_df = pd.DataFrame([[{1:'ds'},{2,'df'}], [{1:'ds'},{2,'df'}]])
test_df.iloc[0,0][1]

最后将经过处理的结构体的列表存储到新的列中，新属性的名称为‘ans’，注意这一步操作已经在初始化函数中完成

In [160]:
dic_temp = data_entity.df.loc[0,'ans'][20]['frame']
dic_temp 

{'level': 'hard',
 'data': {'rotation': [[None, None, 1, 1, 1, 0],
   [2, 2, 0, None, 0, None],
   [None, 2, None, None, 0, None],
   [2, 0, 2, 1, None, None]],
  'lowered': [3, 3]}}

# 添加列（时长），单位s
新增属性'interval'

每一行的实例数据为：

In [217]:
row_data = data_entity.df.loc[0,:]
row_data

user                                          yibin/智能计算素养/310101
type                                                  ticket-user
contest_id                                           yibin/智能计算素养
ticket_id                                                  310101
start_time                              2021-06-24T09:44:29+08:00
expire_time                             2021-06-24T11:44:29+08:00
stop_time                               2021-06-24T10:33:05+08:00
mobile_alert                                                  NaN
task_answers    ["{\"frame\":{\"level\":\"easy\",\"data\":{\"s...
user_agent      Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
client_ip                                          125.69.167.133
school                                                    区二中实验初中
tag                                            我是学生,智能计算素养,问题解决素养
name                                                           杨凤
owner                                                       yibin
state     

In [219]:

start_time = row_data['start_time']
start_time = datetime.strptime(start_time,"%Y-%m-%dT%H:%M:%S+08:00")

expire_time = row_data['expire_time']
expire_time = datetime.strptime(expire_time,"%Y-%m-%dT%H:%M:%S+08:00")

stop_time = row_data['stop_time']
stop_time = datetime.strptime(stop_time,"%Y-%m-%dT%H:%M:%S+08:00")

total_sec = (stop_time - start_time).seconds

In [289]:
total_sec

2916

基于以上思路的代码实现在init函数中执行（self.get_interval）

## 所有答卷的完成时间的分布图如下

In [78]:
data_entity.plot()

0

# 新建关键字是题目的表
（总共23个题目）

每一行中的‘ans’属性为一个固定长度（23）的列表，该列表中的元素格式为词典，现在建立一个新的dataframe，共662行，23列，每列对应同一个题目。

In [19]:
ndf = data_entity.ndf

将ndf切分成23份并提取关键信息

In [20]:
ndf0 = ndf.loc[:,0]
ndf0_df = pd.DataFrame(ndf0)
ndf0_df

,0
0,"{'level': 'easy', 'data': {'successRate': 1, '..."
1,"{'level': 'easy', 'data': {'successRate': 1, '..."
2,"{'level': 'easy', 'data': {'successRate': 1, '..."
3,"{'level': 'easy', 'data': {'successRate': 1, '..."
4,"{'level': 'easy', 'data': {'successRate': 0, '..."
...,...
657,"{'level': 'easy', 'data': {'successRate': 1, '..."
658,"{'level': 'easy', 'data': {'successRate': 1, '..."
659,"{'level': 'easy', 'data': {'successRate': 1, '..."
660,"{'level': 'easy', 'data': {'successRate': 1, '..."


In [1]:
ndf0 = data_entity

NameError: name 'data_entity' is not defined

In [55]:
a = list(data_entity.ndf_list[22].iloc[:,0])
a

[{'level': 'medium',
  'data': {'sequence': [1, 1, 0, 0, 1, 1],
   'transformation': [[0, 1], [1, 0]]}},
 {'level': 'medium',
  'data': {'sequence': [1, 1, 0, 1, 0, 0],
   'transformation': [[0, 1], [1, 0]]}},
 {'level': 'medium',
  'data': {'sequence': [0], 'transformation': [[0], [1, 0, 0]]}},
 {'level': 'medium',
  'data': {'sequence': [0], 'transformation': [[0, 1], [1, 0]]}},
 {'level': 'medium',
  'data': {'sequence': [0], 'transformation': [[0, 1], [1, 1]]}},
 {'level': 'medium',
  'data': {'sequence': [0], 'transformation': [[0, 1, 0], [0, 0]]}},
 {'level': 'medium',
  'data': {'sequence': [0, 1, 0, 1], 'transformation': [[1, 0, 1], [0]]}},
 {'sequence': [0], 'transformation': [[0, 1], [1, 1]]},
 {'level': 'medium',
  'data': {'sequence': [0], 'transformation': [[0, 1], [1, 0]]}},
 {'level': 'medium',
  'data': {'sequence': [0, 0, 0, 0, 0, 0, 0],
   'transformation': [[1, 0], [0, 0, 0]]}},
 {'level': 'medium',
  'data': {'sequence': [0, 0, 0, 0, 0],
   'transformation': [[0, 1,

In [62]:
for i in data_entity.ndf_list:
    df_temp = i
    df_str_list = []
    for j in range(len(df_temp)):
        ndf_index_j = df_temp.iloc[j, 0]
        if ndf_index_j == None:
            df_str_list.append(str(None))
        else:
            df_str_list.append(content_to_str(ndf_index_j))
# df_str_list
# df_temp.insert(1, 'ans_str', df_str_list)
# len(df_str_list)
#     df_temp.insert(1, 'ans_str', df_str_list)

ValueError: cannot insert ans_str, already exists

In [59]:
def content_to_str(data):
    str_data = ''
    if data == None:
        return str(None)
    elif type(data) == type([]):
        return data_to_str(data)
    elif 'data' in data.keys():
        return data_to_str(data['data'])
    else:
        return data_to_str(data)

def data_to_str(data):
    if type(data) == type({}):
        return str(list(data.values()))
    else:
        return str(data)

In [39]:
test = {'sequence': [1, 1, 0, 0, 1, 0, 1],
   'transformation': [[0], [1, 0]]}
test1 = [[0, 0, 0, 0, 0, 0, 0],
   [1, 0, 1, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0],
   [1, 0, 0, 0, 0, 0, 0]]

In [50]:
type(test) == type({})
str(list(test.values()))

'[[1, 1, 0, 0, 1, 0, 1], [[0], [1, 0]]]'

In [43]:
test1

[[0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0]]

题目的字典有三种格式：
1、ndf0.iloc[66,0] == None

2、ndf0.iloc[0,0]
'data':{'successRate': 1, 'minJumps': 2, 'jumps': 2, 'path': [2, 0, 1]}

3、ndf0.iloc[7,0]
{'successRate': 0, 'minJumps': 2, 'jumps': 3, 'path': [1, 2, 3, 4]}

题目之间的data格式不一样：
0，1为：{'successRate': 1, 'minJumps': 2, 'jumps': 2, 'path': [2, 0, 1]}

2，3为：['00', '01', '02']

4为：[[0, 0, 0, 0, 0, 0, 0],
   [1, 0, 1, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0],
   [1, 0, 0, 0, 0, 0, 0]]
   
5为：['B_A', 'C_A']

6为：['B_A', 'C_A', 'G_F', 'D_B', 'E_B']

7为：[[[0, 1], [0, 2], [1, 2]],
   [[0, 4], [0, 5], [1, 5]],
   [[0, 6], [0, 7], [1, 7]],
   [[0, 10], [0, 11], [1, 11]]]
   
8为：[[[1, 0], [2, 0], [3, 0], [3, 1]],
   [[2, 3], [3, 3], [4, 3], [4, 4]],
   [[0, 4], [1, 4], [2, 4], [2, 5]],
   [[0, 1], [1, 1], [2, 1], [2, 2]],
   [[3, 2], [4, 2], [5, 2], [5, 3]]]
   
9为['09_02', '05_06']

10为['v_14_v_6', 'v_2_v_16', 'v_15_v_12']

11，12为[[{'row': 0, 'col': 0},
    {'row': 1, 'col': 2},
    {'row': 2, 'col': 1},
    {'row': 0, 'col': 2},
    {'row': 2, 'col': 4}]]
    
13为 [2, 1]

14为[1, 2, 1, 2, 2, 3]

15为{'stamps': [1, 0, 1, 2], 'start': 1, 'end': 16, 'selected': True}}

16为{'stamps': [2, 1, 0, 1, 0],
   'start': 1,
   'end': 5,
   'selected': False}
   
17为{'stamps': [2, 1, 0, 1], 'start': 1, 'end': 5, 'selected': False}

18为{'rotation': [[1, 0, 1]], 'lowered': [0, 0]}

 {'rotation': [[2, 2, 0]], 'lowered': None}
 
19为{'rotation': [[0, 0, 1, 3, 0, 1]], 'lowered': None}

 {'rotation': [[2, 3, 1, 0, 0, 1]], 'lowered': [0, 2]}
 
20为{'rotation': [[None, None, 1, 1, 1, 0],
    [1, 1, 3, None, 0, None],
    [None, 1, None, None, 0, None],
    [1, 1, 1, 0, None, None]],
   'lowered': None}
   
21为{'sequence': [1, 0, 1, 0, 0, 1, 1],

   'transformation': [[0], [0, 1]]}
   
22为{'sequence': [1, 1, 0, 0, 1, 0, 1],

   'transformation': [[0], [1, 0]]}
   

## 每道题编码种类

In [72]:
# ndf0_group = ndf0_df.groupby(0)
# list(ndf0_group)
gl = data_entity.group_list


In [88]:
list(gl[4])[1][1]


,4,ans_str
5,"{'level': 'easy', 'data': [[0, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],..."
10,"{'level': 'easy', 'data': [[0, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],..."
54,"{'level': 'easy', 'data': [[0, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],..."
123,"{'level': 'easy', 'data': [[0, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],..."
199,"{'level': 'easy', 'data': [[0, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],..."
437,"{'level': 'easy', 'data': [[0, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],..."
629,"{'level': 'easy', 'data': [[0, 0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0],..."


In [77]:
g1.loc[0,1]

,1,ans_str
66,None,None


In [49]:
len(list(gl[6]))

45

In [68]:
data_entity.plot_problem()

0

基于上述求解思路，编写self.group_by()函数以得出各个题目的编码数量

## 每道题正确率